# Conversational RAG

In [1]:
%pip install --upgrade --quiet  langchain langchain-community langchainhub langchain-chroma bs4 unstructured 
%pip install --upgrade --quiet  lark langchain-chroma



Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
import getpass
import os
import langchain
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_community.document_loaders import DirectoryLoader, UnstructuredMarkdownLoader
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_chroma import Chroma
from langchain_text_splitters import MarkdownHeaderTextSplitter, RecursiveCharacterTextSplitter
from langchain.chains import create_retrieval_chain, create_history_aware_retriever
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain import hub
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain.retrievers.self_query.base import SelfQueryRetriever
from langchain.chains.query_constructor.base import AttributeInfo





## 1. Files Loading

In [3]:
path = r"C:\Users\reply\RepoAgent\markdown_docs\repo_agent"
loader = DirectoryLoader(path, glob="./*.md", show_progress=True, loader_cls=UnstructuredMarkdownLoader)
docs = loader.load()

100%|██████████| 9/9 [00:17<00:00,  1.97s/it]


## 2. Split and Preprocess

In [4]:
def split_documents(doc, chunk_size=250, chunk_overlap=30):
    headers_to_split_on = [
         ("#", "Header 1"),
        ("##", "Header 2"),
        ("###", "Header 3"),
    ]

    markdown_splitter = MarkdownHeaderTextSplitter(
        headers_to_split_on=headers_to_split_on, strip_headers=False
    )
    md_header_splits = markdown_splitter.split_text(doc.page_content)
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size, chunk_overlap=chunk_overlap
    )

    # Split
    splits = text_splitter.split_documents(md_header_splits)
   
    return splits

In [5]:
all_splits = []
for doc in docs:
    splits = split_documents(doc)
    for doc_split in splits:
        doc_split.metadata = doc.metadata
    all_splits.extend(splits)


In [6]:
all_splits

[Document(page_content='ClassDef ChangeDetector', metadata={'source': 'C:\\Users\\reply\\RepoAgent\\markdown_docs\\repo_agent\\change_detector.md'}),
 Document(page_content='ChangeDetector: The function of ChangeDetector is to handle file differences and change detection, particularly focusing on identifying changes in files since the last commit, with a potential use of the FileHandler class for accessing the file', metadata={'source': 'C:\\Users\\reply\\RepoAgent\\markdown_docs\\repo_agent\\change_detector.md'}),
 Document(page_content='class for accessing the file system.', metadata={'source': 'C:\\Users\\reply\\RepoAgent\\markdown_docs\\repo_agent\\change_detector.md'}),
 Document(page_content="Attributes:\n- repo_path: The path to the repository where changes are to be detected.\n- repo: An instance of a git repository, initialized with the path provided during the object's creation.  \nCode Description:", metadata={'source': 'C:\\Users\\reply\\RepoAgent\\markdown_docs\\repo_agent

In [7]:
metadata_field_info = [
    AttributeInfo(
        name="source",
        description="Contain the source of the documentation informations",
        type="string",
    )]
document_content_description = "Code documentation"


In [8]:
vectorstore = Chroma.from_documents(documents = all_splits, embedding=OpenAIEmbeddings())
retriever = vectorstore.as_retriever()#search_kwargs={"filter": {"source": all_splits[1].metadata["source"]}})


In [9]:
#vectorstore.get(where={"source": all_splits[0].metadata["source"]})["documents"]

In [10]:
len(all_splits)

1704

In [11]:
all_splits[1700].metadata["source"]

'C:\\Users\\reply\\RepoAgent\\markdown_docs\\repo_agent\\runner.md'

## 3. Prompting

In [12]:
model = ChatOpenAI(model="gpt-3.5-turbo-0125")

In [13]:


retriever = SelfQueryRetriever.from_llm(
    model,
    vectorstore,
    document_content_description,
    metadata_field_info,
)




## 4. Test

In [14]:
from langchain import hub


In [15]:
prompt = hub.pull("rlm/rag-prompt")

C:\Users\reply\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The class `RetrievalQA` was deprecated in LangChain 0.1.17 and will be removed in 0.3.0. Use create_retrieval_chain instead.
  warn_deprecated(


ValidationError: 2 validation errors for RetrievalQA
combine_documents_chain
  field required (type=value_error.missing)
llm
  extra fields not permitted (type=value_error.extra)

In [ ]:
response = retriever.invoke(" document with source 'C:\\Users\\reply\\RepoAgent\\markdown_docs\\repo_agent\\change_detector.md' ")

In [ ]:
response

[Document(page_content='Note:', metadata={'source': 'C:\\Users\\reply\\RepoAgent\\markdown_docs\\repo_agent\\change_detector.md'}),
 Document(page_content='Note:', metadata={'source': 'C:\\Users\\reply\\RepoAgent\\markdown_docs\\repo_agent\\change_detector.md'}),
 Document(page_content='Note:', metadata={'source': 'C:\\Users\\reply\\RepoAgent\\markdown_docs\\repo_agent\\change_detector.md'}),
 Document(page_content='Note:', metadata={'source': 'C:\\Users\\reply\\RepoAgent\\markdown_docs\\repo_agent\\change_detector.md'})]

In [ ]:
def paginate_text(text, page_size=500):
    return [text[i:i + page_size] for i in range(0, len(text), page_size)]


In [ ]:
query = input("Enter your query (or 'exit' to quit): ")
response = conversational_rag_chain.invoke(
    {"input": query},
    config={
        "configurable": {"session_id": "abc123"}
    },  # constructs a key "abc123" in `store`.
)["answer"]
print(f"Question: {query}\n")
paginated_response = paginate_text(response)

for i, page in enumerate(paginated_response):
    print(f"Page {i + 1}/{len(paginated_response)}: {page}")
    if i < len(paginated_response) - 1:
            input("Press Enter to continue to the next page...")

print("\n")

Question: get the source of get_staged_pys 

Page 1/1: I'm sorry, but the source code for the `get_staged_pys` function is not provided in the context given.




In [ ]:
splits[0]

Document(page_content='FunctionDef make_fake_files  \nmake_fake_files: The function of make_fake_files is to prepare the repository for documentation generation by handling unstaged changes and untracked files according to their status in git.', metadata={'source': 'C:\\Users\\reply\\RepoAgent\\markdown_docs\\repo_agent\\runner.md'})

In [ ]:
rag_chain.invoke({"input":"how get_staged_pys works?"})

KeyError: "Input to ChatPromptTemplate is missing variables {'chat_history'}.  Expected: ['chat_history', 'context', 'input'] Received: ['input', 'context']"

In [ ]:
conversational_rag_chain.invoke(
   {"input":"no"},
    config={
        "configurable": {"session_id": "abc123"}
    },  # constructs a key "abc123" in `store`.
)["answer"]

"I don't have the specific information about the markdown file where the behavior of the last_element function is explained."

In [ ]:
rag_chain.invoke({"input":"What file named last does?"})

KeyError: "Input to ChatPromptTemplate is missing variables {'chat_history'}.  Expected: ['chat_history', 'context', 'input'] Received: ['input', 'context']"

In [ ]:
rag_chain.invoke("What last.md does?")

'last.md saves and updates the markdown content of an existing file after making structural changes.'

In [ ]:
rag_chain.invoke("What prova.md does?")

'The prova.md function assigns values to variables a, b, and c, and then returns the values of b and c. It does not take any parameters and is used for version control operations with the git library.'